<a href="https://colab.research.google.com/github/Dannatinjaca/PROGCOM-A/blob/main/picasYfijas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from abc import ABC, abstractmethod
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import random

class JuegoBase(ABC):
    @abstractmethod
    def iniciar_juego(self):
        pass

    @abstractmethod
    def procesar_intento(self, intento):
        pass

class PicasYFijas(JuegoBase):
    def __init__(self):
        self.iniciar_juego()

    def _generar_secreto(self):
        digitos = list("0123456789")
        random.shuffle(digitos)
        return "".join(digitos[:4])

    def iniciar_juego(self):
        self.secreto = self._generar_secreto()
        self.intentos = 0
        self.historial = []

    def procesar_intento(self, intento):
        self.intentos += 1
        fijas = sum(s == i for s, i in zip(self.secreto, intento))
        picas = sum(min(self.secreto.count(d), intento.count(d)) for d in set(intento)) - fijas
        self.historial.append((intento, fijas, picas))
        return fijas, picas

    def juego_finalizado(self):
        return any(f == 4 for _, f, _ in self.historial)

    def obtener_historial(self):
        return self.historial

    def obtener_secreto(self):
        return self.secreto

class InterfazHTML:
    def __init__(self, juego, imagen_url=None):
        self.juego = juego
        self.imagen_url = imagen_url
        self._construir_widgets()

    def _construir_widgets(self):
        self.entrada = widgets.Text(placeholder='Ej: 1234', description='Tu intento:')
        self.boton = widgets.Button(description='Probar', button_style='success')
        self.reiniciar = widgets.Button(description='Jugar de nuevo', button_style='info')
        self.salida = widgets.Output()
        self.boton.on_click(self._al_hacer_click)
        self.reiniciar.on_click(self._reiniciar_juego)

    def _mostrar_historial(self):
        historial = self.juego.obtener_historial()
        html = """
        <style>
            .intento {
                background: white;
                color: black;
                border: 1px solid #ccc;
                border-radius: 8px;
                padding: 8px;
                margin: 6px 0;
                font-family: monospace;
                box-shadow: 2px 2px 5px rgba(0,0,0,0.1);
            }
            .correcto {
                border: 2px solid green;
                background: #eaffea;
            }
        </style>
        <div>
        """
        for intento, fijas, picas in historial:
            clase = "intento correcto" if fijas == 4 else "intento"
            html += f"<div class='{clase}'>🔢 {intento} → <b>{fijas}</b> fijas, <b>{picas}</b> picas</div>"
        html += "</div>"
        return html

    def _al_hacer_click(self, b):
        intento = self.entrada.value.strip()
        with self.salida:
            clear_output()
            if len(intento) != 4 or not intento.isdigit() or len(set(intento)) != 4:
                display(HTML("<b style='color:red;'>❌ Entrada inválida. Usa 4 dígitos distintos.</b>"))
                return
            f, p = self.juego.procesar_intento(intento)
            display(HTML(self._mostrar_historial()))

            if self.juego.juego_finalizado():
                secreto = self.juego.obtener_secreto()
                display(HTML(f"<h3 style='color:green;'>🎉 ¡Ganaste en {self.juego.intentos} intentos! El número era {secreto}</h3>"))
                display(self.reiniciar)
                self.boton.disabled = True
                self.entrada.disabled = True

    def _reiniciar_juego(self, b):
        self.juego.iniciar_juego()
        self._construir_widgets()
        self.mostrar()

    def mostrar(self):
        with self.salida:
            clear_output()
        display(HTML("<h2 style='color:darkblue;'>🎯 Juego Picas y Fijas</h2>"))
        if self.imagen_url:
            display(HTML(f"<img src='{self.imagen_url}' style='max-height:150px;margin-bottom:10px;'>"))
        display(self.entrada, self.boton, self.salida)


imagen_link = "https://cdn-icons-png.flaticon.com/512/1055/1055646.png"

juego = PicasYFijas()
interfaz = InterfazHTML(juego, imagen_url=imagen_link)
interfaz.mostrar()


Text(value='', description='Tu intento:', placeholder='Ej: 1234')

Button(button_style='success', description='Probar', style=ButtonStyle())

Output()